<a href="https://colab.research.google.com/github/j6teng/fullstack-course4/blob/master/DS266_Final_Project_Fake_News_Detect_(model_optimization).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##a. LIBRARY INSTALL AND IMPORT

In [1]:
!pip install pandas numpy scikit-learn torch transformers tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 59.8 MB/s eta 0:00:00


In [3]:
# This cell will authenticate you and mount your Drive in the Colab.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Import all necessary libraries
import re
import nltk
import os
import json
import pandas as pd
import torch
import numpy as np
from sklearn.utils import resample
from transformers import BertTokenizer, BertModel
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [5]:
# check some numpy version
print(f"NumPy version: {np.__version__}")

NumPy version: 2.0.2


#b.DATA

###b1.DATA LOADING

In [6]:
import json
import pandas as pd

# Function to read a JSON file
def load_json_file(file_path):
    """Load a JSON file"""
    with open(file_path, "r", encoding="utf-8") as file:
        return json.load(file)

# Function to process FakeNewsNet data
def process_FakeNewsNet_data(file_paths):
    """Read multiple JSON files and organize them into a DataFrame"""
    data_list = []  # Store all data

    for category, file_path in file_paths.items():
        data = load_json_file(file_path)  # Load JSON data

        for news_id, news_content in data.items():
            if 'text' in news_content:  # Ensure the text field exists
                # Determine the news source
                if "gossipcop" in category:
                    source = "gossipcop"
                elif "politifact" in category:
                    source = "politifact"
                else:
                    source = "unknown"

                # Determine the news type
                if "R" in category:
                    if "HR" in category:
                        label = "human_real" # Human-written real news
                    else:
                        label = "gpt_real"  # AI-generated real news
                elif "F" in category:
                    if "MF" in category:
                        label = "gpt_fake"  # AI-generated fake news
                    else:
                        label = "human_fake"  # Human-written fake news
                else:
                    label = "unknown"

                data_list.append({
                    "id": news_id,
                    "text": news_content["text"],
                    "title": news_content.get("title", ""),  # Some data may not have a title
                    "source": source,  # News source
                    "label": label,  # Real/fake news category
                })

    # Convert to a Pandas DataFrame
    df = pd.DataFrame(data_list)
    return df

# Define paths for the dataset
fake_news_paths = {
    "gossipcop_HF": "/content/drive/MyDrive/DS266/HF.json",
    "gossipcop_HR": "/content/drive/MyDrive/DS266/HR.json",
    "gossipcop_MF": "/content/drive/MyDrive/DS266/MF.json",
    "gossipcop_MR": "/content/drive/MyDrive/DS266/MR.json",
}

# Load and process the data
FakeNewsNet_df = process_FakeNewsNet_data(fake_news_paths)

# Display the first 5 rows
FakeNewsNet_df.head()


,id,text,title,source,label
0,0,✕ Close Meghan Markle and Prince Harry have an...,As it happened: Prince Harry and Meghan Markle...,gossipcop,human_fake
1,1,Kim Kardashian and Kanye West are pulling out ...,Kim & Kanye Install At-Home Panic Room After P...,gossipcop,human_fake
2,2,Prince Harry and Meghan currently live at Kens...,£1.4million spent renovating Prince Harry and ...,gossipcop,human_fake
3,3,They can't get enough of the Biebs on this sho...,Photos from Dancing With the Stars: Special Gu...,gossipcop,human_fake
4,4,Ben Affleck is keeping life with his three kid...,Jennifer Garner ‘Doesn’t Want’ Her Kids Around...,gossipcop,human_fake
